# Setting up a single simulation pair


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import sim_params_cosmo
import sim_params_space
import write_config
import latin_hypercube
import corner

### Define parameter space, and choose one simulation

In [2]:
# get list of simulation parameters
add_slope=False
add_running=False
add_heat_amp=True
add_heat_slo=False
param_space=sim_params_space.SimulationParameterSpace(add_slope=add_slope,add_running=add_running,
                                                      add_heat_amp=add_heat_amp,add_heat_slo=add_heat_slo)

params=param_space.params
for key,param in params.items():
    print(key,param)

Om_star {'ip': 0, 'min_val': 0.955, 'max_val': 0.975, 'z_star': 3.0, 'latex': '$\\Omega_\\star$'}
Delta2_star {'ip': 1, 'min_val': 0.25, 'max_val': 0.45, 'z_star': 3.0, 'kp_Mpc': 0.7, 'latex': '$\\Delta^2_\\star$'}
heat_amp {'ip': 2, 'min_val': 0.5, 'max_val': 1.5, 'latex': '$H_A$'}
z_rei {'ip': 3, 'min_val': 5.5, 'max_val': 15.0, 'latex': '$z_r$'}


In [3]:
Npar=len(params)
param_limits=np.empty([Npar,2])
for key,param in params.items():
    ip=param['ip']
    param_limits[ip][0]=param['min_val']
    param_limits[ip][1]=param['max_val']

In [4]:
# generate Latin hypercube 
nsamples=1
seed=101
cube=latin_hypercube.get_hypercube_samples(param_limits, nsamples, prior_points = None, seed=seed)
sim_params=cube[0]
print(sim_params)

[ 0.965  0.35   1.    10.25 ]


###  Find simulation cosmology, write all relevant files

In [5]:
# make sure simulation folder exists
sim_dir='test_single_pair'+'/'
os.makedirs(sim_dir,exist_ok=True)
# setup cosmology model
cosmo_sim=sim_params_cosmo.cosmo_from_sim_params(param_space,sim_params,verbose=False)
# make a different folder for each simulation in the pair
plus_dir=sim_dir+'/sim_plus/'
os.makedirs(plus_dir,exist_ok=True)
minus_dir=sim_dir+'/sim_minus/'
os.makedirs(minus_dir,exist_ok=True)
# write GenIC and MP-Gadget parameters, for both simulations in pair
write_config.write_genic_file(plus_dir,cosmo_sim,paired=False)
zs=write_config.write_gadget_file(plus_dir,cosmo_sim)
write_config.write_genic_file(minus_dir,cosmo_sim,paired=True)
_=write_config.write_gadget_file(minus_dir,cosmo_sim)  

In [6]:
# construct linear power model and store in JSON format
linP_zs=write_config.write_sim_json_file(sim_dir,param_space,cosmo_sim,zs=zs)